In [1]:
classes=["2","4","7","9"]
# CNNで学習するときの画像のサイズを設定（サイズが大きいと学習に時間がかかる）
img_width, img_height = 150,150#224,224
#バッチサイズ
batch_size = 50
#エポック数
nb_epoch = 20

In [3]:
import tensorflow as tf
from keras.backend import tensorflow_backend
config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
session=tf.Session(config=config)
tensorflow_backend.set_session(session)

ModuleNotFoundError: No module named 'tensorflow'

In [4]:
import tkinter
from tkinter import filedialog
import os
from PIL import Image
import glob

In [2]:
import os
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import numpy as np
import time

ModuleNotFoundError: No module named 'keras'

In [5]:
#GPUテスト
tf.test.gpu_device_name()

'/device:GPU:0'

In [6]:
# 分類するクラス数
nb_classes = len(classes)

# トレーニング用とバリデーション用の画像格納先
train_data_dir = './train_150k'
validation_data_dir = './test'


#トレーニングデータ用の画像数
nb_train_samples = len(os.listdir(train_data_dir+"/"+str(classes[0])+"/"))#round(len(files)*3*0.8)#round(face_num*9*0.8)
print("nb_train_samples:",nb_train_samples)
#バリデーション用の画像数
nb_validation_samples = len(os.listdir(validation_data_dir+"/"+str(classes[0])+"/"))#round(len(files)*3*0.2)#round(face_num*9*0.2)
print("nb_validation_samples:",nb_validation_samples)


nb_train_samples: 150000
nb_validation_samples: 4770


In [7]:
# VGG16のロード。FC層は不要なので include_top=False
input_tensor = Input(shape=(img_width, img_height, 3))
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

# VGG16のFC層の作成
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.25))
top_model.add(Dense(nb_classes, activation='softmax'))

# VGG16とFC層を結合してモデルを作成
vgg_model = Model(input=vgg16.input, output=top_model(vgg16.output))

# VGG16の15層目まで(最後のコンボリューション層と全結合層以外)の部分は重みを固定
for layer in vgg_model.layers[:15]:
    layer.trainable = False

# 多クラス分類を指定
vgg_model.compile(loss='categorical_crossentropy',
          optimizer=optimizers.SGD(lr=1e-3, momentum=0.9),
          metrics=['accuracy'])







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



D:\program\anaconda\envs\p3.6_01\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
  del sys.path[0]


In [8]:
vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [9]:
top_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 1028      
Total params: 2,098,436
Trainable params: 2,098,436
Non-trainable params: 0
_________________________________________________________________


In [11]:
# トレーンング用、バリデーション用データを生成するジェネレータ作成
train_datagen = ImageDataGenerator(rescale=1.0 / 255)
validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [12]:
train_generator = train_datagen.flow_from_directory(
  train_data_dir,
  target_size=(img_width, img_height),
  color_mode='rgb',
  classes=classes,
  class_mode='categorical',
  batch_size=batch_size,
  shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
  validation_data_dir,
  target_size=(img_width, img_height),
  color_mode='rgb',
  classes=classes,
  class_mode='categorical',
  batch_size=batch_size,
  shuffle=True)

Found 600000 images belonging to 4 classes.
Found 19080 images belonging to 4 classes.


In [13]:
from keras.callbacks import ModelCheckpoint
os.makedirs('models', exist_ok=True)
model_checkpoint = ModelCheckpoint(
    filepath=os.path.join('models', 'model_{epoch:02d}_{val_loss:.2f}.h5'),
    monitor='val_loss',
    verbose=1)

In [ ]:
# Fine-tuning
history = vgg_model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    nb_epoch=nb_epoch,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples,
    callbacks=[model_checkpoint])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


D:\program\anaconda\envs\p3.6_01\lib\site-packages\ipykernel_launcher.py:8: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
D:\program\anaconda\envs\p3.6_01\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., callbacks=[<keras.ca..., steps_per_epoch=3000, epochs=20, validation_steps=4770)`
  


Epoch 1/20
3000/3000 [==============================] - 935s 312ms/step - loss: 0.0581 - acc: 0.9792 - val_loss: 0.0085 - val_acc: 0.9973

Epoch 00001: saving model to models\model_01_0.01.h5
Epoch 2/20
3000/3000 [==============================] - 775s 258ms/step - loss: 0.0217 - acc: 0.9924 - val_loss: 0.0057 - val_acc: 0.9983

Epoch 00002: saving model to models\model_02_0.01.h5
Epoch 3/20
3000/3000 [==============================] - 842s 281ms/step - loss: 0.0155 - acc: 0.9948 - val_loss: 0.0055 - val_acc: 0.9982

Epoch 00003: saving model to models\model_03_0.01.h5
Epoch 4/20
3000/3000 [==============================] - 830s 277ms/step - loss: 0.0140 - acc: 0.9951 - val_loss: 0.0046 - val_acc: 0.9988

Epoch 00004: saving model to models\model_04_0.00.h5
Epoch 5/20
3000/3000 [==============================] - 651s 217ms/step - loss: 0.0103 - acc: 0.9964 - val_loss: 0.0044 - val_acc: 0.9989

Epoch 00005: saving model to models\model_05_0.00.h5
Epoch 6/20
3000/3000 [==================

In [ ]:
result_dir='results'
if not os.path.exists(result_dir):
    os.mkdir(result_dir)
vgg_model.save_weights(os.path.join(result_dir,'Final_weight.h5'))
vgg_model.save(os.path.join(result_dir,'Final_model.h5'))

In [ ]:
acc=copy.copy(history.history["acc"])
acc.insert(0,0)
val=copy.copy(history.history["val_acc"])
val.insert(0,0)
print(acc)
print(val)

In [ ]:
# 学習結果を描写
import matplotlib.pyplot as plt
import copy
#acc, val_accのプロット
plt.rcParams["figure.dpi"]=300
plt.plot(acc, lw=0.5,label="train_acc", ls="-", marker="o")
plt.plot(val,lw=0.5, label="test_acc", ls="-", marker="x")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(loc="best")
plt.xlim(0,20)
plt.ylim(0.97,1.0)
plt.xticks(range(21), range(21))
plt.grid(color='gray')
#学習曲線の保存
plt.savefig(result_dir+'/Final.png')
plt.show()


In [ ]:
print(acc)

In [ ]:
acc=[0, 0.9751999997496605, 0.9927173297007879, 0.994222663640976, 0.9949266634384791, 0.9956639991203944, 0.9959506656328837, 0.996271332859993, 0.9965660007794698, 0.9968806680043538, 0.9968653352657953, 0.9970740027825038, 0.9971706692377726, 0.9973713366190592, 0.9974706704219183, 0.9975140041510264, 0.9976140044530233, 0.9977333385547003, 0.9978646716276804, 0.9978613394498825, 0.9979426728487015]

In [ ]:

x=list(range(21))#1~20
y=[0.9210691823899371,0.9578616352201258,0.9642557651991615,0.9656184486373166,0.9714360587002097,0.9719601677148847,0.9784591194968554,0.9783542976939203,0.9682914046121593,0.9806079664570231,0.975,0.980398322851153,0.9803459119496856,0.9768343815513627,0.9770440251572327,0.9800314465408805,0.9821802935010482,0.9765723270440252,0.9772012578616353,0.9798218029350105]
y.insert(0,0)

In [ ]:
print(len(x))
print(len(y))

In [ ]:
import matplotlib.pyplot as plt
import copy
plt.figure()
plt.rcParams["figure.dpi"]=300
plt.plot(acc,label="train_acc", ls="-", marker="o")
plt.plot(x,y, label="test_acc", ls="-", marker="x")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(loc="best")
plt.xlim(0.0,20)
plt.ylim(0.9,1.0)
plt.xticks(range(21), range(21))
plt.grid(color='gray')
#学習曲線の保存
plt.savefig('results/Final_test.png')
plt.show()

